In [1]:
!pip install pydub mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.7 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import IPython.display as ipd
import scipy.io.wavfile as wavfile

from pydub import AudioSegment
from mido import Message, MidiFile, MidiTrack


In [3]:
NOTE_FREQ = {
    "A": 440.00,   # A4
    "C": 261.63,   # C4
    "D": 293.66,   # D4
    "G": 392.00,   # G4
}
DEFAULT_DURATION = 0.5
SR = 44100

In [4]:

# Función que genera la nota usando el sin con duración y amplitud dadas.:
def generate_sine_wave(frequency, duration, sample_rate=44100, amplitude=0.5):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

# Función principal:
def play_notes(note_string, note_frequencies, note_duration=0.5):
    sample_rate = 44100
    combined_wave = np.array([])

    for note in note_string:
        if note in note_frequencies:
            frequency = note_frequencies[note]
            wave = generate_sine_wave(frequency, note_duration, sample_rate)
            combined_wave = np.concatenate((combined_wave, wave))

    # Normalizar a 16-bit
    audio_wave = np.int16(combined_wave * 32767)

    # Regresar un objeto de audio
    return ipd.Audio(audio_wave, rate=sample_rate)

# Para tocar dos al mismo tiempo
def play_notes_together(note_string1, note_string2, note_frequencies1, note_frequencies2, volume1=0.5, volume2=0.5, note_duration=0.5):
    sample_rate = 44100
    combined_wave = np.array([])

    for note1, note2 in zip(note_string1, note_string2):
        wave1 = generate_sine_wave(note_frequencies1.get(note1, 0), note_duration, sample_rate, volume1) if note1 in note_frequencies1 else np.zeros(int(sample_rate * note_duration))
        wave2 = generate_sine_wave(note_frequencies2.get(note2, 0), note_duration, sample_rate, volume2) if note2 in note_frequencies2 else np.zeros(int(sample_rate * note_duration))
        combined_wave = np.concatenate((combined_wave, wave1 + wave2))

    combined_wave = np.int16(combined_wave / np.max(np.abs(combined_wave)) * 32767)
    return ipd.Audio(combined_wave, rate=sample_rate)

# Función para guardar notas a MP3
def save_notes_to_mp3(note_string, note_frequencies, filename='output.mp3', note_duration=0.5):
    sample_rate = 44100
    combined_wave = np.array([])

    for note in note_string:
        if note in note_frequencies:
            frequency = note_frequencies[note]
            wave = generate_sine_wave(frequency, note_duration, sample_rate)
            combined_wave = np.concatenate((combined_wave, wave))

    # Normalizar y convertir a 16-bit
    audio_wave = np.int16(combined_wave / np.max(np.abs(combined_wave)) * 32767)

    # Guardar temporalmente como WAV
    wavfile.write("temp_output.wav", sample_rate, audio_wave)

    # Convertir a MP3 usando pydub
    sound = AudioSegment.from_wav("temp_output.wav")
    sound.export(filename, format="mp3")

    # Eliminar archivo temporal si se desea
    os.remove("temp_output.wav")

    print(f"Archivo MP3 guardado como: {filename}")

# Función para guardar archivos en formato MIDI
def notes_to_midi(note_string, note_frequencies, filename="output.mid", note_duration=0.5, tempo_bpm=120):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Tempo
    ticks_per_beat = mid.ticks_per_beat
    seconds_per_beat = 60 / tempo_bpm
    ticks_per_note = int((note_duration / seconds_per_beat) * ticks_per_beat)

    # Mapeo de nombre de nota a número MIDI aproximado (asumimos octava 4 por simplicidad)
    def frequency_to_midi_note(freq):
        from math import log2
        return int(round(69 + 12 * log2(freq / 440.0)))

    for note in note_string:
        if note in note_frequencies:
            midi_note = frequency_to_midi_note(note_frequencies[note])
            track.append(Message('note_on', note=midi_note, velocity=64, time=0))
            track.append(Message('note_off', note=midi_note, velocity=64, time=ticks_per_note))

    mid.save(filename)
    print(f"Archivo MIDI guardado como: {filename}")

  # Esta función es para guardar en MP3 el audio generado por play notes together
def save_notes_together_to_mp3(note_string1, note_string2, note_frequencies1, note_frequencies2,
                               filename='output_together.mp3', volume1=0.5, volume2=0.5, note_duration=0.5):
    sample_rate = 44100
    combined_wave = np.array([])

    for note1, note2 in zip(note_string1, note_string2):
        wave1 = generate_sine_wave(note_frequencies1.get(note1, 0), note_duration, sample_rate, volume1) if note1 in note_frequencies1 else np.zeros(int(sample_rate * note_duration))
        wave2 = generate_sine_wave(note_frequencies2.get(note2, 0), note_duration, sample_rate, volume2) if note2 in note_frequencies2 else np.zeros(int(sample_rate * note_duration))
        combined_wave = np.concatenate((combined_wave, wave1 + wave2))

    # Normalizar
    audio_wave = np.int16(combined_wave / np.max(np.abs(combined_wave)) * 32767)

    # Guardar como WAV temporal
    wavfile.write("temp_output_together.wav", sample_rate, audio_wave)

    # Convertir a MP3
    sound = AudioSegment.from_wav("temp_output_together.wav")
    sound.export(filename, format="mp3")
    os.remove("temp_output_together.wav")

    print(f"Archivo MP3 guardado como: {filename}")
# Esta función es para guardar el audio en formato MIDI
def notes_together_to_midi(note_string1, note_string2, note_frequencies1, note_frequencies2,
                           filename="output_together.mid", note_duration=0.5, tempo_bpm=120):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    ticks_per_beat = mid.ticks_per_beat
    seconds_per_beat = 60 / tempo_bpm
    ticks_per_note = int((note_duration / seconds_per_beat) * ticks_per_beat)

    def frequency_to_midi_note(freq):
        from math import log2
        return int(round(69 + 12 * log2(freq / 440.0)))

    for note1, note2 in zip(note_string1, note_string2):
        if note1 in note_frequencies1:
            midi_note1 = frequency_to_midi_note(note_frequencies1[note1])
            track.append(Message('note_on', note=midi_note1, velocity=64, time=0))
        if note2 in note_frequencies2:
            midi_note2 = frequency_to_midi_note(note_frequencies2[note2])
            track.append(Message('note_on', note=midi_note2, velocity=64, time=0))

        # Apagar notas después del tiempo correspondiente
        if note1 in note_frequencies1:
            track.append(Message('note_off', note=midi_note1, velocity=64, time=ticks_per_note))
        if note2 in note_frequencies2:
            track.append(Message('note_off', note=midi_note2, velocity=64, time=0))

    mid.save(filename)
    print(f"Archivo MIDI guardado como: {filename}")


In [5]:
def play_user_string(user_string,
                     note_duration=DEFAULT_DURATION,
                     note_freqs=NOTE_FREQ):
    """
    Clean and validate the raw string, then call play_notes().
    """
    cleaned = user_string.strip().upper().replace(" ", "")
    bad = [ch for ch in cleaned if ch not in note_freqs]
    if bad:
        raise ValueError("Only A, C, D, G allowed – wrong chars: " + ", ".join(bad))
    return play_notes(cleaned, note_freqs, note_duration)

In [6]:
import ipywidgets as w
from IPython.display import display, clear_output

text_in   = w.Text(value="ACDG", description="Sequence:", layout=w.Layout(width="180px"))
dur_slider = w.FloatSlider(value=DEFAULT_DURATION, min=0.1, max=1.0,
                           step=0.05, description="Dur (s):")
play_btn  = w.Button(description="Play!", button_style="success")
out_area  = w.Output()

def _on_play(_):
    with out_area:
        clear_output(wait=True)
        try:
            audio = play_user_string(text_in.value, dur_slider.value)
            display(audio)
        except ValueError as e:
            print(e)

play_btn.on_click(_on_play)

display(w.VBox([w.HBox([text_in, dur_slider, play_btn]), out_area]))